In [45]:
import os
import polars as pl
pl.Config(tbl_rows=100)

In [46]:
df = pl.read_parquet(os.path.join("data","cnb_ceska_proza.parquet")).rename({'autorstvo_kod':'100_7'})
df = pl.concat([df, pl.read_parquet(os.path.join("data","cnb_ceska_poezie.parquet"))])
spisovatelstvo = list(set(df.select(pl.col('100_7')).drop_nulls().to_series().to_list()))
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.filter(pl.col("100_7").is_in(spisovatelstvo))
aut = pl.read_parquet(os.path.join("data","aut_vyber.parquet"))

In [47]:
print(len(aut))

1222398


In [48]:
aut = aut.explode("370_c").filter(pl.col("370_c").str.contains("Česk")).explode("100_7").select(["100_7","374_a","375_a","678_a"])
df = df.join(aut, left_on="100_7", right_on="100_7", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")

In [49]:
print(len(aut))

363993


In [50]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))

df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

794838
711787


In [51]:
df = df.drop_nulls(subset=["700_7", "700_4", "700_a"]).explode(["700_7", "700_4", "700_a"])

In [67]:
import json

In [71]:
with open(os.path.join("data","marc_zkratky.json"),"r",encoding="utf-8") as f:
    zkratky = json.loads(f.read())

In [75]:
df = df.with_columns(pl.col("700_4").replace(zkratky))

In [77]:
df.group_by(['700_a','700_7']).agg(pl.col('700_4').unique().len()).sort(by='700_4',descending=True)

700_a,700_7,700_4
str,str,u32
"""Werich, Jan,""","""jk01151601""",14
"""Svolinský, Karel,""","""jk01122138""",14
"""Martinková, Věra,""","""jn20000401743""",13
"""Bouda, Cyril,""","""jk01012795""",13
"""Dyrynk, Martin,""","""jk01030272""",12
"""Brunner, Vratislav Hugo,""","""jk01013327""",11
"""Teige, Karel,""","""jk01132144""",11
"""Hrubín, František,""","""jk01042920""",10
"""Seifert, Jaroslav,""","""jk01110657""",10


In [79]:
df.filter(pl.col('700_7') == "jk01122138").group_by('700_4').len().sort(by='len',descending=True)

700_4,len
str,u32
"""illustrator""",180
"""artist""",7
"""typographer""",5
"""book designer""",4
"""lithographer""",2
"""author of introduction, etc.""",1
"""engraver""",1
"""wood engraver""",1
"""bookjacket designer""",1


In [81]:
df.filter(pl.col('700_7') == "jk01151601").group_by('700_4').len().sort(by='len',descending=True)

700_4,len
str,u32
"""author""",63
"""author of introduction, etc.""",4
"""adapter""",2
"""correspondent""",2
"""bibliographic antecedent""",2
"""author of afterword, colophon,…",2
"""editor""",1
"""lyricist""",1
"""contributor""",1


In [83]:
df.filter(pl.col('700_7') == "jn20000401743").group_by('700_4').len().sort(by='len',descending=True)

700_4,len
str,u32
"""redaktor""",35
"""illustrator""",34
"""editor""",32
"""book designer""",14
"""artist""",6
"""bookjacket designer""",5
"""author""",4
"""designer""",2
"""photographer""",2


In [85]:
df.filter(pl.col('700_7') == "jk01012795").group_by('700_4').len().sort(by='len',descending=True)

700_4,len
str,u32
"""illustrator""",213
"""book designer""",8
"""wood engraver""",7
"""artist""",4
"""lithographer""",4
"""bookjacket designer""",3
"""binding designer""",2
"""typographer""",2
"""other""",2


In [87]:
df.filter(pl.col('700_7') == "jx20100301005").group_by('700_4').len().sort(by='len',descending=True)

700_4,len
str,u32
"""translator""",3
